In [ ]:
# Necessary imports
import pandas as pd

### Function to load datasets and handle errors. 
**Main Features**
- File type check: Ensures the file is a CSV.
- Data loading: Attempts to read the CSV using Pandas.
- Empty data check: Raises an error if the file or dataset is empty.
- Separator check: Detects potential wrong separators if only one column is found.
- Error handling: Handles common errors like file not found, empty files, parsing issues, and encoding problems.

In [327]:
def load_dataset(df_name, sep=','):

    """
    Load a dataset from a CSV file, with error handling for common issues like file not found,
    empty files, incorrect separators, and encoding problems.
    
    """
    try:
        # Check if the file is a CSV
        if not df_name.endswith('.csv'):
            raise ValueError("Only CSV files are supported.")
        
        # Try to load the dataset
        df = pd.read_csv(df_name)
        
        # Check if the dataframe is empty
        if df.empty:
            raise ValueError(f"The dataset '{df_name}' is empty.")
        
        # Check if the file was read correctly (if only 1 column, it's likely the wrong separator)
        if len(df.columns) == 1:
            raise ValueError(f"The file '{df_name}' might have an incorrect separator. Currently using '{sep}'.")
        
        df.columns = df.columns.str.lower()
        
        return df
    
    except FileNotFoundError:
        raise FileNotFoundError(f"The file '{df_name}' was not found. Please check the file path.")
    
    except pd.errors.EmptyDataError:
        raise ValueError(f"The file '{df_name}' is empty or contains only headers.")
    
    except pd.errors.ParserError:
        raise ValueError(f"There was an error parsing the file '{df_name}'. It might be corrupted.")
    
    except UnicodeDecodeError:
        raise ValueError(f"Failed to decode the file '{df_name}'. It might have an unsupported encoding.")
    
    except Exception as e:
        raise ValueError(f"An unexpected error occurred while loading the dataset: {e}")


In [350]:
# This cell will need update, because I don't know how the data will be named
shop_sales = load_dataset('./data/shop_sales.csv')
shop_sales_dates = load_dataset('./data/shop_sales_dates.csv')
shop_sales_prices = load_dataset('./data/shop_sales_prices.csv')

### Function to fill missing values in shop_sales_dataset. 
**Main Features**
- Column type conversion: Converts date_id, item_id, store_id, and cnt to appropriate data types.
- Handling NaN in date_id: Fills missing date_id values incrementally while checking for consecutive NaNs and raises an error if found.
- Filling NaNs in item_id: Fills missing item_id values based on surrounding rows.
- Filling NaNs in store_id: Uses the prefix of item_id to fill missing store_id values.
- Final check for NaNs: Warns if any NaN values remain after filling.

In [342]:
def fill_missing_values_shop_sales(initial_df):
    """
    Fills missing values (NaNs) in the 'date_id', 'item_id', and 'store_id' columns of the dataset
    based on specific rules provided. Also converts column types at the beginning and handles 
    consecutive NaNs in the 'date_id' column.

    """
    df = initial_df.reset_index(drop=True)


    # Convert column types
    df['date_id'] = pd.to_numeric(df['date_id'], errors='coerce').astype(float)  # Allowing NaNs as float
    df['item_id'] = df['item_id'].astype(str)
    df['store_id'] = df['store_id'].astype(str)
    df['cnt'] = pd.to_numeric(df['cnt'], errors='coerce').astype(float)

    # Sort data by date_id to ensure correct sequential logic
    # df = df.sort_values(by=['item_id']).reset_index(drop=True)

    # Get the maximum date_id
    max_date_id = df['date_id'].max() 

    for i in range(1, len(df)):
        if pd.isna(df.at[i, 'date_id']):
            # Detect consecutive NaNs in 'date_id'
            if pd.isna(df.at[i - 1, 'date_id']):
                raise ValueError(f"Consecutive NaN values detected in 'date_id' at index {i}. Cannot determine the correct value.")
            # Handle non-consecutive NaN
            if df.at[i - 1, 'date_id'] == max_date_id:
                df.at[i, 'date_id'] = 1
            else:
                df.at[i, 'date_id'] = df.at[i - 1, 'date_id'] + 1

    # Fill NaN values in 'item_id'
    for i in range(len(df)):
        if pd.isna(df.at[i, 'item_id']):
            if df.at[i, 'date_id'] == 1:
                df.at[i, 'item_id'] = df.at[i + 1, 'item_id'] if i + 1 < len(df) else df.at[i - 1, 'item_id']
            elif df.at[i, 'date_id'] == max_date_id:
                df.at[i, 'item_id'] = df.at[i - 1, 'item_id']
            else:
                df.at[i, 'item_id'] = df.at[i - 1, 'item_id']

    # Fill NaN values in 'store_id' based on 'item_id'
    for i in range(len(df)):
        if pd.isna(df.at[i, 'store_id']):
            df.at[i, 'store_id'] = df.at[i, 'item_id'].split('_')[0]  # Extract the prefix from item_id

    # Final check if there are any remaining NaN values
    if df.isna().sum().sum() > 0:
        print("Warning: Some NaN values are still present after filling.")

    return df

In [343]:
#Example usage
shop_sales.at[1, 'date_id'] = float('NaN')
shop_sales_processed = fill_missing_values_shop_sales(shop_sales)
shop_sales_processed.at[1, 'date_id']

2.0

### Function to fill missing values in shop_sales_dates_dataset
**Main Features**
- Column type conversion: Converts date_id, wm_yr_wk, wday, month, year, and cashback_store_* columns to appropriate numeric types, allowing NaNs where necessary.
- Filling NaNs in date_id: Interpolates missing date_id values and sets the first and last dates manually.
- Drop weekday column: Removes the unused weekday column from the dataset.
- Filling NaNs in date: Fills missing date values sequentially, raising an error if consecutive NaNs are detected.
- Extract date components: Fills missing wday, month, and year values using the date column.
- Handle missing event data: Replaces NaNs in event_name_1, event_type_1, event_name_2, and event_type_2 with 'Unknown'.
- Handle missing cashback values: Fills NaNs in cashback_store_1, cashback_store_2, and cashback_store_3 with 0.

In [346]:
def preprocess_shop_sales_dates(initial_df):
    """
    Filling NaNs and converting types based on the following rules:
    - Convert data types.
    - Fill NaNs in 'date' and 'date_id' in the same way as 'date_id' in the previous dataset.
    - Drop 'weekday' column.
    - If 'wday', 'month', or 'year' is NaN, extract it from 'date'.
    - Replace NaNs in 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2' with 'Unknown'.
    - Fill NaNs in 'cashback_store_1', 'cashback_store_2', 'cashback_store_3' with 0.
    """
    df = initial_df.reset_index(drop=True)
    # Convert column types
    df['date_id'] = pd.to_numeric(df['date_id'], errors='coerce')
    df['wm_yr_wk'] = pd.to_numeric(df['wm_yr_wk'], errors='coerce').astype(int)
    df['wday'] = pd.to_numeric(df['wday'], errors='coerce').astype(int)
    df['month'] = pd.to_numeric(df['month'], errors='coerce').astype(int)
    df['year'] = pd.to_numeric(df['year'], errors='coerce').astype(int)
    df['cashback_store_1'] = pd.to_numeric(df['cashback_store_1'], errors='coerce').astype(float)
    df['cashback_store_2'] = pd.to_numeric(df['cashback_store_2'], errors='coerce').astype(float)
    df['cashback_store_3'] = pd.to_numeric(df['cashback_store_3'], errors='coerce').astype(float)

    df['date_id'] = df['date_id'].interpolate(method='linear')

    df.at[0, 'date_id'] = 1
    df.loc[df.index[-1], 'date_id'] = df['date_id'].iloc[-2] + 1

    # Drop 'weekday' column
    df = df.drop(columns=['weekday'])

    if pd.isna(df.at[0, 'date']):
        if not pd.isna(df.at[1, 'date']):
            df.at[0, 'date'] = df.at[1, 'date'] - pd.DateOffset(days=1)
        else:
            raise ValueError(f"Consecutive NaN values detected in 'date' at index 1. Cannot determine the correct value.")

    # Fill NaNs in 'date' in the same way as 'date_id'
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    for i in range(1, len(df)):
        if pd.isna(df.at[i, 'date']):
            if pd.isna(df.at[i - 1, 'date']):
                raise ValueError(f"Consecutive NaN values detected in 'date' at index {i}. Cannot determine the correct value.")
            df.at[i, 'date'] = df.at[i - 1, 'date'] + pd.DateOffset(days=1)

    # Fill 'wday', 'month', 'year' from 'date'
    df['wday'] = df['wday'].fillna(df['date'].dt.dayofweek + 1)  # wday is 1-based
    df['month'] = df['month'].fillna(df['date'].dt.month)
    df['year'] = df['year'].fillna(df['date'].dt.year)

    # Replace NaNs in event columns with 'Unknown'
    event_cols = ['event_name_1', 'event_type_1', 'event_name_2', 'event_type_2']
    df[event_cols] = df[event_cols].fillna('Unknown')

    # Fill NaNs in cashback_store columns with 0
    cashback_cols = ['cashback_store_1', 'cashback_store_2', 'cashback_store_3']
    df[cashback_cols] = df[cashback_cols].fillna(0)

    # Final check for remaining NaN values
    # if df.isna().sum().sum() > 0:
    #     print("Warning: Some NaN values are still present after filling.")

    return df

In [347]:
# Example usage
shop_sales_dates.at[0, 'date_id'] = float('NaN')
shop_sales_dates_processed = preprocess_shop_sales_dates(shop_sales_dates)
shop_sales_dates_processed

,date,wm_yr_wk,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,date_id,cashback_store_1,cashback_store_2,cashback_store_3
0,2011-01-29,11101,1,1,2011,Unknown,Unknown,Unknown,Unknown,1.0,0.0,0.0,0.0
1,2011-01-30,11101,2,1,2011,Unknown,Unknown,Unknown,Unknown,2.0,0.0,0.0,0.0
2,2011-01-31,11101,3,1,2011,Unknown,Unknown,Unknown,Unknown,3.0,0.0,0.0,0.0
3,2011-02-01,11101,4,2,2011,Unknown,Unknown,Unknown,Unknown,4.0,0.0,1.0,1.0
4,2011-02-02,11101,5,2,2011,Unknown,Unknown,Unknown,Unknown,5.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1814,2016-01-17,11551,2,1,2016,Unknown,Unknown,Unknown,Unknown,1815.0,0.0,0.0,0.0
1815,2016-01-18,11551,3,1,2016,MartinLutherKingDay,National,Unknown,Unknown,1816.0,0.0,0.0,0.0
1816,2016-01-19,11551,4,1,2016,Unknown,Unknown,Unknown,Unknown,1817.0,0.0,0.0,0.0
1817,2016-01-20,11551,5,1,2016,Unknown,Unknown,Unknown,Unknown,1818.0,0.0,0.0,0.0


### Function to fill missing values in shop_sales_dates_dataset
**Main Features**
- Filling NaNs in item_id: Populates missing item_id values based on the surrounding rows, using logic similar to previous datasets.
- Filling NaNs in store_id: Fills missing store_id by extracting the prefix from item_id.
- Filling wm_yr_wk values: Uses linear interpolation to fill missing wm_yr_wk values, rounding the result to the nearest integer.
- Sorting: Sorts the dataset by item_id and wm_yr_wk to ensure proper data order.
- Interpolating sell_price: Fills missing sell_price values using linear interpolation after sorting.

In [351]:
def preprocess_shop_sales_prices(initial_df):
    """
    Preprocesses the sales dataset by filling NaNs and sorting the data based on specific rules:
    - Fill NaNs in 'item_id' and 'store_id' like in the first dataset.
    - Fill 'wm_yr_wk' using linear interpolation and round if it's a float.
    - Sort the DataFrame by ['item_id', 'wm_yr_wk'] and apply interpolation where needed.
    """
    df = initial_df.reset_index(drop=True)

    df['wm_yr_wk'] = pd.to_numeric(df['wm_yr_wk'], errors='coerce').astype(float)
    df['sell_price'] = pd.to_numeric(df['sell_price'], errors='coerce').astype(float)

    # Fill NaN values in 'item_id' as done in the first dataset
    max_item_id = df['item_id'].max()
    for i in range(1, len(df)):
        if pd.isna(df.at[i, 'item_id']):
            if df.at[i, 'wm_yr_wk'] == 1:
                df.at[i, 'item_id'] = df.at[i + 1, 'item_id'] if i + 1 < len(df) else df.at[i - 1, 'item_id']
            elif df.at[i, 'wm_yr_wk'] == max_item_id:
                df.at[i, 'item_id'] = df.at[i - 1, 'item_id']
            else:
                df.at[i, 'item_id'] = df.at[i - 1, 'item_id']

    # Fill NaN values in 'store_id' based on 'item_id'
    for i in range(len(df)):
        if pd.isna(df.at[i, 'store_id']):
            df.at[i, 'store_id'] = df.at[i, 'item_id'].split('_')[0]  # Extract the prefix from item_id

    # Fill 'wm_yr_wk' with linear interpolation and round
    df['wm_yr_wk'] = df['wm_yr_wk'].interpolate(method='linear').round(0)

    # Sort by 'item_id' and 'wm_yr_wk'
    df = df.sort_values(by=['item_id', 'wm_yr_wk']).reset_index(drop=True)

    # Interpolate 'sell_price' if needed after sorting
    df['sell_price'] = df['sell_price'].interpolate(method='linear')

    return df

In [352]:
shop_sales_prices.at[0, 'wm_yr_wk'] = float('NaN')
shop_sales_prices_processed = preprocess_shop_sales_prices(shop_sales_prices)
shop_sales_prices_processed

,store_id,item_id,wm_yr_wk,sell_price
0,STORE_1,STORE_1_064,11101.0,2.54
1,STORE_1,STORE_1_064,11102.0,2.54
2,STORE_1,STORE_1_064,11103.0,2.54
3,STORE_1,STORE_1_064,11104.0,2.54
4,STORE_1,STORE_1_064,11105.0,2.54
...,...,...,...,...
11136,STORE_3,STORE_3_804,11547.0,1.88
11137,STORE_3,STORE_3_804,11548.0,1.88
11138,STORE_3,STORE_3_804,11549.0,1.88
11139,STORE_3,STORE_3_804,11550.0,1.88


### Function to merge datasets 
**Main Features**
- Lowercase column names: Ensures all column names are lowercase for consistency across datasets.
- Validation of date_id: Checks if the date_id column exists in both shop_sales and shop_sales_dates before merging.
- Merging datasets: Merges shop_sales and shop_sales_dates on the date_id column using an outer join.
- Create unique identifier: Generates a unique item_id_wm_yr_wk column in both datasets to facilitate price mapping.
- Duplicate check: Verifies that there are no duplicate values in the item_id_wm_yr_wk column of the shop_sales_prices dataset.
- Map prices: Maps the sell_price from shop_sales_prices to the merged dataset using item_id_wm_yr_wk.
- Error handling: Catches and raises errors related to missing columns, duplicates, or other unexpected issues.

In [365]:
def safe_merge(shop_sales, shop_sales_dates, shop_sales_prices):
    """
    Merge two datasets on the 'date_id' column after ensuring both datasets contain this column,
    create 'item_id_wm_yr_wk' columns, handle potential errors, check for duplicates,
    and check for new NaN values in the merged DataFrame.
    """

    try:
        # Make column names lowercase
        shop_sales.columns = shop_sales.columns.str.lower()
        shop_sales_dates.columns = shop_sales_dates.columns.str.lower()
        shop_sales_prices.columns = shop_sales_prices.columns.str.lower()

        # Check if 'date_id' exists in both DataFrames
        if 'date_id' not in shop_sales.columns:
            raise ValueError("'date_id' column is missing in 'shop_sales' dataset.")
        if 'date_id' not in shop_sales_dates.columns:
            raise ValueError("'date_id' column is missing in 'shop_sales_dates' dataset.")
        
        merged_df = pd.merge(shop_sales, shop_sales_dates, how='outer', left_on='date_id', right_on='date_id')
        
        old_nans = shop_sales.isna().sum().sum()
        new_nans = merged_df.isna().sum().sum()
        if new_nans - old_nans > 0:
            print(f"Warning: {new_nans - old_nans} new NaN values appeared while merging shop sales and dates datasets.")

        # Create 'item_id_wm_yr_wk' in both datasets - a column with unique values
        merged_df['item_id_wm_yr_wk'] = merged_df.item_id.astype(str) + '_' + merged_df.wm_yr_wk.astype(float).astype(str)
        shop_sales_prices['item_id_wm_yr_wk'] = shop_sales_prices.item_id.astype(str) + '_' + shop_sales_prices.wm_yr_wk.astype(float).astype(str)

        # Check for duplicates in 'item_id_wm_yr_wk' in shop_sales_prices
        if shop_sales_prices.duplicated('item_id_wm_yr_wk').any():
            raise ValueError("Duplicates found in 'item_id_wm_yr_wk' column in 'shop_sales_prices' dataset.")

        # Map prices
        map_prices_dict = dict(tuple(zip(shop_sales_prices.item_id_wm_yr_wk, shop_sales_prices.sell_price)))
        merged_df['sell_price'] = merged_df.item_id_wm_yr_wk.map(map_prices_dict)

        old_nans = new_nans
        new_nans = merged_df.isna().sum().sum()
        if new_nans - old_nans > 0:
            print(f"Warning: {new_nans - old_nans} new NaN values appeared while mapping sell prices on merged dataset.")

        merged_df.sort_values(by=['item_id', 'date_id'], inplace=True)
        merged_df.drop(columns=['item_id_wm_yr_wk'], inplace=True)

        return merged_df
    
    except ValueError as e:
        raise ValueError(f"Error in merging process: {e}")
    
    except Exception as e:
        raise Exception(f"An unexpected error occurred: {e}")


In [366]:
#Example usage on datasets before cleaning NaNs
merged_df = safe_merge(shop_sales, shop_sales_dates, shop_sales_prices)

In [367]:
#Example usage on datasets after cleaning NaNs
merged_df = safe_merge(shop_sales_processed, shop_sales_dates_processed, shop_sales_prices_processed)

In [368]:
merged_df

,item_id,store_id,date_id,cnt,date,wm_yr_wk,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,cashback_store_1,cashback_store_2,cashback_store_3,sell_price
30,STORE_1_064,STORE_1,1.0,0.0,2011-01-29,11101,1,1,2011,Unknown,Unknown,Unknown,Unknown,0.0,0.0,0.0,2.54
75,STORE_1_064,STORE_1,2.0,1.0,2011-01-30,11101,2,1,2011,Unknown,Unknown,Unknown,Unknown,0.0,0.0,0.0,2.54
120,STORE_1_064,STORE_1,3.0,0.0,2011-01-31,11101,3,1,2011,Unknown,Unknown,Unknown,Unknown,0.0,0.0,0.0,2.54
165,STORE_1_064,STORE_1,4.0,0.0,2011-02-01,11101,4,2,2011,Unknown,Unknown,Unknown,Unknown,0.0,1.0,1.0,2.54
210,STORE_1_064,STORE_1,5.0,0.0,2011-02-02,11101,5,2,2011,Unknown,Unknown,Unknown,Unknown,1.0,1.0,0.0,2.54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81659,STORE_3_804,STORE_3,1815.0,21.0,2016-01-17,11551,2,1,2016,Unknown,Unknown,Unknown,Unknown,0.0,0.0,0.0,1.88
81704,STORE_3_804,STORE_3,1816.0,28.0,2016-01-18,11551,3,1,2016,MartinLutherKingDay,National,Unknown,Unknown,0.0,0.0,0.0,1.88
81749,STORE_3_804,STORE_3,1817.0,5.0,2016-01-19,11551,4,1,2016,Unknown,Unknown,Unknown,Unknown,0.0,0.0,0.0,1.88
81794,STORE_3_804,STORE_3,1818.0,8.0,2016-01-20,11551,5,1,2016,Unknown,Unknown,Unknown,Unknown,0.0,0.0,0.0,1.88
